# ANALYSIS

In [1]:
import pandas as pd
import ast
import numpy as np

# Load the csv file

In [15]:
# df = pd.read_csv('best-books.csv',index_col='book_index')
df = pd.read_csv('best-books_old.csv')
df.head()

,url,title,author,num_reviews,num_ratings,avg_rating,num_pages,original_publish_year,series,genres,awards,places
0,https://www.goodreads.com/book/show/2767052-th...,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,179510,6995195,4.32,284,2008,True,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Panem']"
1,https://www.goodreads.com/book/show/2.Harry_Po...,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,48128,2781662,4.50,870,2003,True,NaN,['Bram Stoker Award for Works for Young Reader...,"['Hogwarts School of Witchcraft and Wizardry',..."
2,https://www.goodreads.com/book/show/2657.To_Ki...,To Kill a Mockingbird,Harper Lee,98678,4978320,4.27,324,1960,False,NaN,"['Pulitzer Prize for Fiction (1961)', 'Audie A...","['Maycomb, Alabama', 'Alabama']"
3,https://www.goodreads.com/book/show/1885.Pride...,Pride and Prejudice,Jane Austen,79238,3368553,4.27,279,1813,False,NaN,NaN,"['United Kingdom', 'Derbyshire, England', 'Eng..."
4,https://www.goodreads.com/book/show/41865.Twil...,"Twilight (The Twilight Saga, #1)",Stephenie Meyer,110648,5441218,3.62,387,2005,True,NaN,"['Georgia Peach Book Award (2007)', 'Buxtehude...","['Forks, Washington', 'Phoenix, Arizona', 'Was..."


# Change the awards col into num of awards:

In [24]:
# df2 = df.head(50)
df = df.replace({'awards': {np.nan: 0}})

d = []

for i in df.index:
    if df.loc[i, "awards"] == 0:
        d.append(0)
    else:
        d.append( len( ast.literal_eval(df.loc[i, "awards"]) ) )

df['awards length'] = d
# df2['awards length'].sum()

# min max normalization

In [18]:
df_min_max = df['avg_rating'].copy()
def min_max_normalization(df_min_max):
    df_min_max = 1 + (df_min_max - df_min_max.min()) / (df_min_max.max() - df_min_max.min()) * 9
    return round(df_min_max, 2)

minmax_norm_ratings = min_max_normalization(df_min_max)
# print(minmax_norm_ratings)
df['minmax_norm_ratings'] = minmax_norm_ratings
print(df)


                                                    url  \
0     https://www.goodreads.com/book/show/2767052-th...   
1     https://www.goodreads.com/book/show/2.Harry_Po...   
2     https://www.goodreads.com/book/show/2657.To_Ki...   
3     https://www.goodreads.com/book/show/1885.Pride...   
4     https://www.goodreads.com/book/show/41865.Twil...   
...                                                 ...   
1342  https://www.goodreads.com/book/show/50623864-t...   
1343  https://www.goodreads.com/book/show/17617277-real   
1344  https://www.goodreads.com/book/show/38743.All_...   
1345  https://www.goodreads.com/book/show/15858248-w...   
1346  https://www.goodreads.com/book/show/7933292-he...   

                                                  title           author  \
0               The Hunger Games (The Hunger Games, #1)  Suzanne Collins   
1     Harry Potter and the Order of the Phoenix (Har...     J.K. Rowling   
2                                 To Kill a Mockingbird       H

# mean normalization

In [19]:
def mean_normalization(df_min_max):
    df_min_max = 1 + (df_min_max - df_min_max.mean()) / (df_min_max.max() - df_min_max.min()) * 9
    return round(df_min_max, 2)

mean_norm_ratings = mean_normalization(df_min_max)
df['mean_norm_ratings'] = mean_norm_ratings
# print(df2)

# sort by minmax_norm_ratings Given Author

In [20]:
def check_author(df, author):
    df = df.loc[df.author == author]
    df.sort_values(by=['minmax_norm_ratings'], ascending=[False])
    return df

check_author(df, 'J.K. Rowling')

,url,title,author,num_reviews,num_ratings,avg_rating,num_pages,original_publish_year,series,genres,awards,places,minmax_norm_ratings,mean_norm_ratings


# sort by publishing year and get mean of minmax_norm_ratings

In [7]:
df.groupby('original_publish_year')['minmax_norm_ratings'].mean()

original_publish_year
180     5.980000
632     6.720000
800     4.920000
900     1.950000
1001    3.330000
          ...   
2017    6.202500
2018    6.665385
2019    6.295714
2020    6.846667
2021    7.300000
Name: minmax_norm_ratings, Length: 199, dtype: float64

# sort by author and check who has highest average rating

In [26]:
# df.groupby('author')['minmax_norm_ratings'].mean().sort_values( ascending=False)
df.groupby('author')['awards length'].sum().sort_values( ascending=False)

author
Stephen King     38
J.K. Rowling     36
Neil Gaiman      31
John Green       27
Rick Riordan     25
                 ..
Kate Chopin       0
Katie McGarry     0
Katy Evans        0
Ken Kesey         0
A.A. Milne        0
Name: awards length, Length: 713, dtype: int64

# sort by genre and check ratings

In [14]:
# NEED TO SEE HOW THIS BEHAVES WITH 3 VALUES=3 GENRES
df.groupby('genres')['minmax_norm_ratings'].mean().sort_values( ascending=False)

Series([], Name: minmax_norm_ratings, dtype: float64)

# sort no of awards by author

In [15]:
# df['awards length']
df.sort_values(by=['awards length', 'author'], ascending=[False, True])

,url,title,author,num_reviews,num_ratings,avg_rating,num_pages,original_publish_year,series,genres,awards,places,awards length,minmax_norm_ratings,mean_norm_ratings
564,https://www.goodreads.com/book/show/3591262-cu...,Cutting for Stone,Abraham Verghese,26226,352881,4.29,541,2009,False,NaN,['PEN/Hemingway Foundation Award Nominee (2010...,"['Addis Ababa', 'Madras', 'Kochi']",5,6.35,2.15
144,https://www.goodreads.com/book/show/472331.Wat...,Watchmen,Alan Moore,15136,510630,4.37,416,1987,False,NaN,"['Hugo Award for Other Forms (1988)', 'Locus A...","['New York City, New York']",5,6.77,2.57
265,https://www.goodreads.com/book/show/546018.Roots,Roots: The Saga of an American Family,Alex Haley,3758,155050,4.42,888,1976,False,NaN,"['ASJA Outstanding Book Award (1978)', 'Audie ...","['The United States of America', 'Gambia']",5,7.04,2.84
49,https://www.goodreads.com/book/show/12232938-t...,The Lovely Bones,Alice Sebold,41998,2139648,3.83,372,2002,False,NaN,['Bram Stoker Award for Best First Novel (2002...,"['Philadelphia, Pennsylvania', 'Pennsylvania',...",5,3.91,-0.29
434,https://www.goodreads.com/book/show/7735333-ma...,"Matched (Matched, #1)",Ally Condie,33954,709214,3.65,369,2010,True,NaN,['South Carolina Book Award for Young Adult (2...,NaN,5,2.96,-1.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,https://www.goodreads.com/book/show/28407.Germ...,Germinal,Émile Zola,1524,31498,4.16,592,1885,False,NaN,0,['France'],0,5.66,1.46
453,https://www.goodreads.com/book/show/30528544,العصبي,أحمد جمال الدين رمضان,150,2261,4.55,165,2016,False,NaN,0,NaN,0,7.72,3.53
454,https://www.goodreads.com/book/show/30528535,انتحار فاشل,أحمد جمال الدين رمضان,201,2616,4.54,175,2016,False,NaN,0,NaN,0,7.67,3.47
322,https://www.goodreads.com/book/show/40937505,زندگی مه آلود پریا,سید مرتضی مصطفوی,134,940,3.90,204,2015,False,NaN,0,NaN,0,4.28,0.09


# sort by minmax_norm_ratings Given Year of publication

In [16]:
def check_year(df, year):
    df = df.loc[df.original_publish_year == year]
    df.sort_values(by=['minmax_norm_ratings'], ascending=[False])
    return df

# check_year(df, 2005)

# check if book series are rated better than non series books

In [35]:
# df.groupby('series')['minmax_norm_ratings'].mean()
# df.groupby('series').size()

df[ (df['num_ratings'] < df['num_ratings'].std()) & (df['avg_rating']>4) ].sort_values(by=['num_ratings'], ascending=[True])

,url,title,author,num_reviews,num_ratings,avg_rating,num_pages,original_publish_year,series,genres,awards,places,minmax_norm_ratings,mean_norm_ratings,places_new,awards length
1123,https://www.goodreads.com/book/show/17889664-t...,"To Live and Die in Fantasyland (To Live, #3)",Ben Peller,78,91,4.29,320,2013,True,NaN,0,0,6.35,2.15,0,0
1023,https://www.goodreads.com/book/show/17889664-t...,"To Live and Die in Fantasyland (To Live, #3)",Ben Peller,78,91,4.29,320,2013,True,NaN,0,0,6.35,2.15,0,0
1220,https://www.goodreads.com/book/show/57213211-d...,"Desert Outcast (Sovereign, #0.5)",Juddy Anderson C. Punzalan,23,194,4.63,340,2021,True,NaN,0,0,8.15,3.95,0,0
1112,https://www.goodreads.com/book/show/56236618-t...,"The Archon of Peace (Sovereign, #1)",Juddy Anderson C. Punzalan,51,229,4.61,795,2020,True,NaN,0,0,8.04,3.84,0,0
1212,https://www.goodreads.com/book/show/56236618-t...,"The Archon of Peace (Sovereign, #1)",Juddy Anderson C. Punzalan,51,229,4.61,795,2020,True,NaN,0,0,8.04,3.84,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,https://www.goodreads.com/book/show/345627.Vam...,"Vampire Academy (Vampire Academy, #1)",Richelle Mead,23304,599819,4.11,332,2007,True,NaN,0,['Montana'],5.39,1.20,[Montana],0
826,https://www.goodreads.com/book/show/36072.The_...,The 7 Habits of Highly Effective People: Power...,Stephen R. Covey,11891,604200,4.13,372,1988,False,NaN,0,0,5.50,1.30,0,0
111,https://www.goodreads.com/book/show/2187.Middl...,Middlesex,Jeffrey Eugenides,23975,606838,4.01,529,2002,False,NaN,"['Pulitzer Prize for Fiction (2003)', 'James T...","['Bursa', 'Izmir', 'Detroit, Michigan']",4.86,0.67,"[Bursa, Izmir, Detroit, Michigan]",5
317,https://www.goodreads.com/book/show/33.The_Lor...,The Lord of the Rings,J.R.R. Tolkien,12445,607011,4.50,1216,1955,False,NaN,['Hugo Award Nominee for Best All-Time Series ...,['Middle-earth'],7.46,3.26,[Middle-earth],5


# WORK IN PROGRESS - check what place is more commonly reffered to in these books

In [22]:
# df2 = df2.replace({'places': {'nan': 0}})
df = df.replace({'places': {np.nan: 0}})
print(df['places'])

dictt = []
for i in df.index:
    if df.loc[i, "places"] == 0:
        dictt.append(0)
    else:
        dictt.append( ast.literal_eval(df.loc[i, "places"]) )

df['places_new'] = dictt
# print(df['places_new'])




# df.groupby('places').size()

0       ['District 12, Panem', 'Capitol, Panem', 'Panem']
1       ['Hogwarts School of Witchcraft and Wizardry',...
2                         ['Maycomb, Alabama', 'Alabama']
3       ['United Kingdom', 'Derbyshire, England', 'Eng...
4       ['Forks, Washington', 'Phoenix, Arizona', 'Was...
                              ...                        
1342                ['France', 'New York City, New York']
1343                                                    0
1344             ['United Kingdom', 'Yorkshire, England']
1345                        ['San Francisco, California']
1346                                                    0
Name: places, Length: 1347, dtype: object


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))

plt.title('', fontsize=16)
plt.xlim([-40,40])
plt.xlabel('')
plt.ylabel('')
plt.legend()
plt.savefig('.png')